# Baseline Experiment - Load Pre-optimized Solution

This notebook loads the best pre-optimized solution from snapshots, validates it, and computes the score.

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
from shapely.strtree import STRtree
from decimal import Decimal, getcontext
import json

getcontext().prec = 30

# Tree shape definition (from getting-started notebook)
def get_tree_polygon():
    """Create the Christmas tree polygon shape."""
    trunk_w = 0.15
    trunk_h = 0.2
    base_w = 0.7
    mid_w = 0.4
    top_w = 0.25
    tip_y = 0.8
    
    # Vertices of the tree (15 points)
    vertices = [
        (-trunk_w/2, 0),           # trunk bottom left
        (-trunk_w/2, trunk_h),     # trunk top left
        (-base_w/2, trunk_h),      # base left
        (-mid_w/2, trunk_h + 0.2), # mid left outer
        (-base_w/2 + 0.1, trunk_h + 0.2), # mid left inner
        (-top_w/2, trunk_h + 0.4), # top left outer
        (-mid_w/2 + 0.1, trunk_h + 0.4), # top left inner
        (0, tip_y),                # tip
        (mid_w/2 - 0.1, trunk_h + 0.4), # top right inner
        (top_w/2, trunk_h + 0.4),  # top right outer
        (mid_w/2, trunk_h + 0.2),  # mid right inner
        (base_w/2 - 0.1, trunk_h + 0.2), # mid right outer
        (base_w/2, trunk_h),       # base right
        (trunk_w/2, trunk_h),      # trunk top right
        (trunk_w/2, 0),            # trunk bottom right
    ]
    return Polygon(vertices)

TREE_POLY = get_tree_polygon()
print(f"Tree polygon created with {len(TREE_POLY.exterior.coords)} vertices")
print(f"Tree bounds: {TREE_POLY.bounds}")
print(f"Tree area: {TREE_POLY.area:.6f}")

In [ ]:
def parse_s_value(s_val):
    """Parse a string value prefixed with 's' to float."""
    if isinstance(s_val, str) and s_val.startswith('s'):
        return float(s_val[1:])
    return float(s_val)

def load_submission(path):
    """Load submission CSV and parse values."""
    df = pd.read_csv(path)
    df['x_val'] = df['x'].apply(parse_s_value)
    df['y_val'] = df['y'].apply(parse_s_value)
    df['deg_val'] = df['deg'].apply(parse_s_value)
    
    # Extract n from id (format: NNN_T)
    df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
    df['tree_idx'] = df['id'].apply(lambda x: int(x.split('_')[1]))
    
    return df

# Load baseline submission
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/submission/submission.csv'
df = load_submission(baseline_path)
print(f"Loaded {len(df)} rows")
print(f"N values range: {df['n'].min()} to {df['n'].max()}")
print(f"Expected rows: {sum(range(1, 201))} = 20100")
df.head(10)

In [ ]:
def create_tree_at_position(x, y, deg):
    """Create a tree polygon at the given position and rotation."""
    # First rotate around origin, then translate
    tree = rotate(TREE_POLY, deg, origin=(0, 0))
    tree = translate(tree, x, y)
    return tree

def get_bounding_box_side(polygons):
    """Get the side length of the smallest square bounding box."""
    if not polygons:
        return 0
    
    all_coords = []
    for poly in polygons:
        all_coords.extend(list(poly.exterior.coords))
    
    xs = [c[0] for c in all_coords]
    ys = [c[1] for c in all_coords]
    
    width = max(xs) - min(xs)
    height = max(ys) - min(ys)
    
    return max(width, height)

def check_overlaps(polygons):
    """Check for overlapping polygons using STRtree."""
    if len(polygons) <= 1:
        return []
    
    overlaps = []
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx > i:  # Only check each pair once
                if polygons[i].intersects(polygons[idx]):
                    if not polygons[i].touches(polygons[idx]):
                        # Check if it's a real overlap (not just touching)
                        intersection = polygons[i].intersection(polygons[idx])
                        if intersection.area > 1e-10:  # Small tolerance
                            overlaps.append((i, idx, intersection.area))
    
    return overlaps

print("Functions defined successfully")

In [ ]:
# Compute score and validate for all n
scores_by_n = {}
overlap_errors = []

for n in range(1, 201):
    n_df = df[df['n'] == n]
    
    if len(n_df) != n:
        print(f"ERROR: n={n} has {len(n_df)} trees, expected {n}")
        continue
    
    # Create polygons
    polygons = []
    for _, row in n_df.iterrows():
        poly = create_tree_at_position(row['x_val'], row['y_val'], row['deg_val'])
        polygons.append(poly)
    
    # Check for overlaps
    overlaps = check_overlaps(polygons)
    if overlaps:
        overlap_errors.append((n, overlaps))
    
    # Compute bounding box side
    side = get_bounding_box_side(polygons)
    
    # Compute score contribution: s^2 / n
    score_n = (side ** 2) / n
    scores_by_n[n] = {'side': side, 'score': score_n}
    
    if n <= 10 or n % 20 == 0:
        print(f"n={n:3d}: side={side:.6f}, score_contribution={score_n:.6f}")

print(f"\nProcessed all {len(scores_by_n)} configurations")
print(f"Overlap errors found: {len(overlap_errors)}")
if overlap_errors:
    for n, overlaps in overlap_errors[:5]:
        print(f"  n={n}: {len(overlaps)} overlaps")

In [ ]:
# Compute total score
total_score = sum(s['score'] for s in scores_by_n.values())
print(f"\n{'='*50}")
print(f"TOTAL SCORE: {total_score:.6f}")
print(f"{'='*50}")
print(f"\nTarget score: 68.888293")
print(f"Gap to target: {total_score - 68.888293:.6f}")
print(f"Percentage improvement needed: {(total_score - 68.888293) / total_score * 100:.2f}%")

In [ ]:
# Analyze score contributions by n-range
print("\nScore breakdown by n-range:")
print("-" * 40)

ranges = [(1, 10), (11, 20), (21, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(scores_by_n[n]['score'] for n in range(start, end+1))
    print(f"n={start:3d}-{end:3d}: {range_score:.6f}")

print("-" * 40)
print(f"Total: {total_score:.6f}")

In [ ]:
# Copy submission to output folder
import shutil

# Copy the baseline submission
shutil.copy(baseline_path, '/home/submission/submission.csv')
print("Copied baseline submission to /home/submission/submission.csv")

# Also save to experiment folder
shutil.copy(baseline_path, '/home/code/experiments/000_baseline/submission.csv')
print("Copied to experiment folder")

# Verify the copy
df_verify = pd.read_csv('/home/submission/submission.csv')
print(f"Verified: {len(df_verify)} rows in submission")

In [ ]:
# Save metrics
metrics = {
    'cv_score': total_score,
    'total_score': total_score,
    'overlap_errors': len(overlap_errors),
    'target': 68.888293,
    'gap': total_score - 68.888293
}

with open('/home/code/experiments/000_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("Saved metrics.json")
print(json.dumps(metrics, indent=2))